# Create PCRGLOBWB forcing Sophie

In [1]:
import shutil
import subprocess
import os
from glob import glob
from pathlib import Path
import shutil

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import yaml
import pandas as pd

from cartopy import crs
from cartopy import feature as cfeature
from cartopy.io import shapereader

import esmvaltool

## Set Paths

In [2]:
FLUXDIR = Path('/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/')
MODELDIR = Path('/lustre1/0/wtrcycle/lorentz-workshop/pcr-globwb/input/')
flux_selection_file = 'fluxnet_toren_selectie_klein_Sophie_Jaren.csv'

## Load dataframes

In [3]:
# Load FLUXNET selection DataFrame
df = pd.read_csv(f'{FLUXDIR}/{flux_selection_file}', sep=';')
df = df.set_index('SITE_ID')

df_extents = pd.read_csv(f'{FLUXDIR}/clonemap_extents.csv')
df_extents = df_extents.set_index('site_id')

## Extract start years & end years

In [4]:
start_years = list(df['Vanaf'])
start_years = [x[:4] for x in start_years]
end_years = list(df['Tot'])
end_years = [x[:4] for x in end_years]

df_extents['start_year'] = start_years
df_extents['end_year'] = end_years

## Create ESMValTooL recipes

In [5]:
def derive_esmvaltool_recipe(station_name, lon_min, lon_max, lat_min, lat_max, start_year, end_year):

    # def derive_esmvaltool_recipe(station, start_year, end_year):
    """Update specific sections in ESMValTool recipe and save as new file."""
    esmvaltool_root = esmvaltool.__path__[0]
    template = f"{FLUXDIR}/recipe_pcrglobwb_sophie.yml"

    with open(template) as f:
        recipe = yaml.load(f, Loader=yaml.CLoader)

    # Update the basin name for the diagnostic script
    recipe["diagnostics"]["diagnostic_daily"]["scripts"]["script"][
        "basin"
    ] = station_name.lower()

    # Set the extents of desired the forcing data
    
    esmvaltool_padding = 1.5
    
    recipe["preprocessors"]["crop_basin"]["extract_region"].update(
        {
            "start_latitude": lat_min - esmvaltool_padding,
            "end_latitude": lat_max + esmvaltool_padding,
            "start_longitude": lon_min - esmvaltool_padding,
            "end_longitude": lon_max + esmvaltool_padding,
        }
    )

    for var in ("tas", "pr", "evspsblpot"):
        recipe["diagnostics"]["diagnostic_daily"]["variables"][var][
            "start_year"
        ] = start_year
        recipe["diagnostics"]["diagnostic_daily"]["variables"][var][
            "end_year"
        ] = end_year

    output_file = f"/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_{station_name}.yml"
    with open(output_file, "w") as f:
        yaml.dump(recipe, f, default_flow_style=False)
    
    return output_file

In [6]:
all_recipes = []
for station_name, row in df_extents.iterrows():
    print(station_name)
    lon_min = float(row.lon_min)
    lon_max = float(row.lon_max)
    lat_min = float(row.lat_min)
    lat_max = float(row.lat_max)
    
    start_year = int(row.start_year) - 1
    end_year = int(row.end_year)
    
    new_recipe_file = derive_esmvaltool_recipe(station_name, lon_min, lon_max, lat_min, lat_max, start_year, end_year)
    all_recipes.append(new_recipe_file)

AU-Cpr
BR-Npw
CN-Ha2
ES-LgS
FR-Pue
GF-Guy
MY-PSO
NL-Loo
SD-Dem
US-ARM
US-Ha1
US-Me2
US-MMS
ZA-Kru
ZM-Mon


## Run esmvaltool

In [7]:
for recipe in all_recipes:
    print(recipe)
    subprocess.check_call(["esmvaltool", "run", recipe])

/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_AU-Cpr.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_BR-Npw.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_CN-Ha2.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_ES-LgS.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_FR-Pue.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_GF-Guy.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_MY-PSO.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_NL-Loo.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_SD-Dem.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recipe_pcrglobwb_US-ARM.yml
/lustre1/0/wtrcycle/users/jaerts/camels_wflow/sophie_BEP/recipes/recip

## Copy output to model directory

In [8]:
for station_name, row in df_extents.iterrows():
    
    # Copy paste output
    forcing_files = glob(f"/lustre1/0/wtrcycle/users/jaerts/camels_wflow/forcing/output/*{station_name}*/work/diagnostic_daily/script/*.nc")\
    
    for forcing in forcing_files:
        
        OUTDIR = f'{MODELDIR}/global_05min/meteo/SophieBEP/'
        outfile = os.path.basename(forcing) 
        shutil.copy(forcing, f'{OUTDIR}/{outfile}')

#     shutil.rmtree(glob(f"/lustre1/0/wtrcycle/users/jaerts/camels_wflow/forcing/output/*{station_name}*/"))